# Cheeger Constant Computation for K₇

## Motivation

The eigenvalue pipeline is **k-dependent**:
- k = 0.366 × √N → λ₁×H* = 14
- k = 0.74 × √N → λ₁×H* = 13

Without calibration, we cannot distinguish 13 from 14.

**Solution**: Compute the Cheeger constant h(K₇) directly!

## Cheeger Inequality

For compact Riemannian manifolds:
$$\lambda_1 \geq \frac{h^2}{4}$$

where h is the **Cheeger constant** (isoperimetric ratio):
$$h(M) = \inf_S \frac{\text{Area}(\partial S)}{\min(\text{Vol}(S), \text{Vol}(M \setminus S))}$$

For **graphs**, the discrete Cheeger constant is:
$$h(G) = \min_{S \subset V, |S| \leq |V|/2} \frac{|E(S, V \setminus S)|}{|S|}$$

## Key Question

Is **h(K₇) × H*** a topological constant?

If h × H* ≈ 2√13 or 2√14, that would confirm the spectral gap formula.

---

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
import json
from datetime import datetime
import time
import gc

# GIFT Constants
H_STAR = 99
DIM_G2 = 14
DET_G = 65/32
RATIO = H_STAR / 84

print("GIFT Constants:")
print(f"  H* = {H_STAR}")
print(f"  dim(G₂) = {DIM_G2}")
print(f"  det(g) = {DET_G}")
print(f"\nCheeger predictions:")
print(f"  If λ₁×H* = 13: h×H* = 2√13 ≈ {2*np.sqrt(13):.2f}")
print(f"  If λ₁×H* = 14: h×H* = 2√14 ≈ {2*np.sqrt(14):.2f}")

In [ ]:
# K₇ Sampling (TCS representation)

def sample_TCS_K7(N, seed=42):
    """Sample K₇ via Twisted Connected Sum: S¹ × S³ × S³"""
    rng = np.random.default_rng(seed)
    
    # S¹ component
    theta = rng.uniform(0, 2*np.pi, N).astype(np.float32)
    
    # S³ components (unit quaternions)
    def sample_S3(n):
        x = rng.standard_normal((n, 4)).astype(np.float32)
        return x / np.linalg.norm(x, axis=1, keepdims=True)
    
    q1 = sample_S3(N)
    q2 = sample_S3(N)
    
    return theta, q1, q2


def compute_K7_distances_sparse(theta, q1, q2, k_neighbors):
    """
    Compute sparse kNN graph for K₇.
    Returns weight matrix W (sparse) and degree vector.
    """
    N = len(theta)
    alpha = DET_G / (RATIO ** 3)
    
    # Compute kNN for each point in batches
    batch_size = 3000
    all_neighbors = []
    all_distances = []
    
    for i_start in range(0, N, batch_size):
        i_end = min(i_start + batch_size, N)
        batch_size_actual = i_end - i_start
        
        # Distance from batch to all points
        D_batch = np.zeros((batch_size_actual, N), dtype=np.float32)
        
        for i_local, i_global in enumerate(range(i_start, i_end)):
            # S¹ distance
            diff_theta = np.abs(theta[i_global] - theta)
            d_S1 = np.minimum(diff_theta, 2*np.pi - diff_theta)
            
            # S³ distances (quaternion)
            dot1 = np.clip(np.abs(np.sum(q1[i_global] * q1, axis=1)), -1, 1)
            d_S3_1 = 2 * np.arccos(dot1)
            
            dot2 = np.clip(np.abs(np.sum(q2[i_global] * q2, axis=1)), -1, 1)
            d_S3_2 = 2 * np.arccos(dot2)
            
            # Combined TCS metric
            D_batch[i_local] = np.sqrt(alpha * d_S1**2 + d_S3_1**2 + RATIO**2 * d_S3_2**2)
        
        # Get k nearest neighbors (excluding self)
        k_actual = min(k_neighbors + 1, N)
        idx = np.argpartition(D_batch, k_actual, axis=1)[:, :k_actual]
        dists = np.take_along_axis(D_batch, idx, axis=1)
        
        # Sort and exclude self (distance 0)
        sort_idx = np.argsort(dists, axis=1)
        idx = np.take_along_axis(idx, sort_idx, axis=1)[:, 1:k_neighbors+1]
        dists = np.take_along_axis(dists, sort_idx, axis=1)[:, 1:k_neighbors+1]
        
        all_neighbors.append(idx)
        all_distances.append(dists)
        
        del D_batch
        gc.collect()
    
    neighbors = np.vstack(all_neighbors)
    distances = np.vstack(all_distances)
    
    # Bandwidth
    sigma = float(np.median(distances[:, -1]))
    
    # Build sparse weight matrix
    row = np.repeat(np.arange(N), k_neighbors)
    col = neighbors.flatten()
    weights = np.exp(-distances.flatten()**2 / (2 * sigma**2))
    
    W = csr_matrix((weights, (row, col)), shape=(N, N))
    W = (W + W.T) / 2  # Symmetrize
    
    return W, sigma

print("✓ K₇ sampling functions defined")

In [ ]:
# Cheeger Constant Computation

def compute_cheeger_spectral(W):
    """
    Compute Cheeger constant using SPECTRAL METHOD.
    
    The Fiedler vector (eigenvector of λ₂ of normalized Laplacian)
    gives a near-optimal cut via its sign pattern.
    
    Cheeger's inequality: h ≤ √(2λ₁) (upper bound from spectral gap)
    """
    N = W.shape[0]
    
    # Degree
    d = np.array(W.sum(axis=1)).flatten()
    d_inv_sqrt = 1.0 / np.sqrt(d + 1e-10)
    
    # Normalized Laplacian
    D_inv_sqrt = csr_matrix((d_inv_sqrt, (range(N), range(N))), shape=(N, N))
    L_norm = csr_matrix(np.eye(N)) - D_inv_sqrt @ W @ D_inv_sqrt
    
    # Compute Fiedler vector (2nd smallest eigenvector)
    eigvals, eigvecs = eigsh(L_norm.tocsr(), k=3, which='SM', tol=1e-8)
    idx = np.argsort(eigvals)
    lambda1 = eigvals[idx[1]]  # Second smallest (first non-zero)
    fiedler = eigvecs[:, idx[1]]
    
    # Sweep cut using Fiedler vector
    # Sort vertices by Fiedler value
    order = np.argsort(fiedler)
    
    # Try all possible cuts and find minimum conductance
    best_h = float('inf')
    
    # Weights crossing the cut
    W_dense = W.toarray() if W.nnz < N * N / 10 else None
    
    # Vol(S) = sum of degrees in S
    vol_total = d.sum()
    
    # Incremental computation
    S_vertices = set()
    vol_S = 0
    cut_weight = 0  # Total weight of edges crossing the cut
    
    for i in range(N // 2):  # Only consider cuts with |S| ≤ N/2
        v = order[i]
        S_vertices.add(v)
        vol_S += d[v]
        
        # Update cut weight: edges from v to vertices not in S
        if W_dense is not None:
            cut_weight += W_dense[v, :].sum() - sum(W_dense[v, u] for u in S_vertices)
            cut_weight -= sum(W_dense[u, v] for u in S_vertices if u != v)
        else:
            # Sparse version
            row_v = W.getrow(v).toarray().flatten()
            for u in S_vertices:
                cut_weight -= row_v[u] if u != v else 0
            cut_weight += row_v.sum() - sum(row_v[u] for u in S_vertices)
        
        # Conductance: cut_weight / min(vol_S, vol_total - vol_S)
        if vol_S > 0 and vol_S < vol_total:
            min_vol = min(vol_S, vol_total - vol_S)
            h = cut_weight / min_vol
            if h < best_h:
                best_h = h
    
    return best_h, lambda1


def compute_cheeger_random_sampling(W, n_samples=1000, seed=42):
    """
    Estimate Cheeger by random sampling of cuts.
    Gives an UPPER BOUND on h.
    """
    rng = np.random.default_rng(seed)
    N = W.shape[0]
    d = np.array(W.sum(axis=1)).flatten()
    vol_total = d.sum()
    
    W_csr = W.tocsr()
    
    best_h = float('inf')
    
    for _ in range(n_samples):
        # Random subset S with size ≤ N/2
        size = rng.integers(1, N // 2 + 1)
        S = set(rng.choice(N, size=size, replace=False))
        
        # Compute cut weight and volumes
        cut_weight = 0
        vol_S = sum(d[v] for v in S)
        
        for v in S:
            row = W_csr.getrow(v)
            for j, w in zip(row.indices, row.data):
                if j not in S:
                    cut_weight += w
        
        min_vol = min(vol_S, vol_total - vol_S)
        if min_vol > 0:
            h = cut_weight / min_vol
            best_h = min(best_h, h)
    
    return best_h


print("✓ Cheeger computation functions defined")

In [ ]:
# Simplified Cheeger via Spectral Bound

def cheeger_from_eigenvalue(lambda1):
    """
    Cheeger-Buser bounds:
    
    h² / 4 ≤ λ₁ ≤ 2h  (for normalized Laplacian with h small)
    
    This gives:
    √(2λ₁) ≥ h ≥ √(λ₁/2)  (approximately)
    
    For graphs: h ≈ √(2λ₁) is often a good approximation.
    """
    h_upper = np.sqrt(2 * lambda1)  # Cheeger upper bound
    h_lower = lambda1 / 2  # From h²/4 ≤ λ₁, so h ≥ 2√λ₁... wait
    # Actually: λ₁ ≥ h²/4 → h ≤ 2√λ₁
    # And λ₁ ≤ 2h (Buser) → h ≥ λ₁/2
    return h_lower, h_upper


print("Cheeger bounds from eigenvalue:")
print(f"  λ₁ = 13/99 → h ∈ [{13/99/2:.4f}, {np.sqrt(2*13/99):.4f}]")
print(f"  λ₁ = 14/99 → h ∈ [{14/99/2:.4f}, {np.sqrt(2*14/99):.4f}]")
print(f"\n  h × H* (lower): 13 case → {13/99/2 * 99:.2f}, 14 case → {14/99/2 * 99:.2f}")
print(f"  h × H* (upper): 13 case → {np.sqrt(2*13/99) * 99:.2f}, 14 case → {np.sqrt(2*14/99) * 99:.2f}")

In [ ]:
# Main computation

print("=" * 70)
print("CHEEGER CONSTANT COMPUTATION FOR K₇")
print("=" * 70)

# Parameters
N_VALUES = [2000, 5000, 10000]
K_VALUES = [50, 100, 150]  # Different k to see stability

results = []

for N in N_VALUES:
    print(f"\nN = {N}")
    print("-" * 50)
    
    # Sample K₇
    theta, q1, q2 = sample_TCS_K7(N, seed=42)
    
    for k in K_VALUES:
        print(f"  k = {k}...", end=" ")
        t0 = time.time()
        
        # Build graph
        W, sigma = compute_K7_distances_sparse(theta, q1, q2, k)
        
        # Compute λ₁ directly
        d = np.array(W.sum(axis=1)).flatten()
        d_inv_sqrt = 1.0 / np.sqrt(d + 1e-10)
        D_inv_sqrt = csr_matrix((d_inv_sqrt, (range(N), range(N))), shape=(N, N))
        L_norm = csr_matrix(np.eye(N)) - D_inv_sqrt @ W @ D_inv_sqrt
        
        eigvals, _ = eigsh(L_norm.tocsr(), k=3, which='SM', tol=1e-8)
        lambda1 = sorted(eigvals)[1]
        
        # Cheeger bounds from λ₁
        h_lower, h_upper = cheeger_from_eigenvalue(lambda1)
        
        # Products
        lambda1_H = lambda1 * H_STAR
        h_lower_H = h_lower * H_STAR
        h_upper_H = h_upper * H_STAR
        
        elapsed = time.time() - t0
        
        print(f"λ₁×H* = {lambda1_H:.2f}, h×H* ∈ [{h_lower_H:.2f}, {h_upper_H:.2f}] [{elapsed:.1f}s]")
        
        results.append({
            'N': int(N),
            'k': int(k),
            'sigma': float(sigma),
            'lambda1': float(lambda1),
            'lambda1_H': float(lambda1_H),
            'h_lower': float(h_lower),
            'h_upper': float(h_upper),
            'h_lower_H': float(h_lower_H),
            'h_upper_H': float(h_upper_H)
        })
        
        del W, L_norm
        gc.collect()

In [ ]:
# Analysis: Is h×H* stable across k?

print("\n" + "=" * 70)
print("STABILITY ANALYSIS")
print("=" * 70)

for N in N_VALUES:
    subset = [r for r in results if r['N'] == N]
    print(f"\nN = {N}:")
    print(f"  {'k':>5} | {'λ₁×H*':>8} | {'h_lower×H*':>12} | {'h_upper×H*':>12}")
    print(f"  {'-'*50}")
    
    for r in subset:
        print(f"  {r['k']:>5} | {r['lambda1_H']:>8.2f} | {r['h_lower_H']:>12.2f} | {r['h_upper_H']:>12.2f}")
    
    # Spread
    l1_values = [r['lambda1_H'] for r in subset]
    h_upper_values = [r['h_upper_H'] for r in subset]
    
    print(f"  Spread λ₁×H*: {max(l1_values) - min(l1_values):.2f}")
    print(f"  Spread h_upper×H*: {max(h_upper_values) - min(h_upper_values):.2f}")

In [ ]:
# Key insight: Cheeger bounds

print("\n" + "=" * 70)
print("KEY INSIGHT: CHEEGER BOUNDS")
print("=" * 70)

print("""
The Cheeger inequality λ₁ ≥ h²/4 gives us:

If we can show h(K₇) ≥ c/H* for some constant c,
then λ₁ ≥ c²/(4H*²), i.e., λ₁×H* ≥ c²/(4H*)

From our measurements:
""")

# Use high-N, middle-k result
ref_result = [r for r in results if r['N'] == max(N_VALUES) and r['k'] == 100]
if ref_result:
    r = ref_result[0]
    print(f"  At N={r['N']}, k={r['k']}:")
    print(f"    λ₁ × H* = {r['lambda1_H']:.2f}")
    print(f"    h_upper × H* = {r['h_upper_H']:.2f}")
    print(f"")
    print(f"  If h_upper ≈ √(2λ₁) is accurate:")
    print(f"    h × H* = √(2 × {r['lambda1_H']:.2f}) = {np.sqrt(2 * r['lambda1_H']):.2f}")
    print(f"")
    print(f"  Expected from theory:")
    print(f"    If λ₁×H* = 13: √(2×13) = {np.sqrt(26):.2f}")
    print(f"    If λ₁×H* = 14: √(2×14) = {np.sqrt(28):.2f}")

In [ ]:
# Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: λ₁×H* vs k for each N
ax = axes[0]
for N in N_VALUES:
    subset = [r for r in results if r['N'] == N]
    ks = [r['k'] for r in subset]
    l1s = [r['lambda1_H'] for r in subset]
    ax.plot(ks, l1s, 'o-', label=f'N={N}')

ax.axhline(14, color='red', linestyle='--', alpha=0.7, label='Pell (14)')
ax.axhline(13, color='blue', linestyle='--', alpha=0.7, label='Spinor (13)')
ax.set_xlabel('k (neighbors)')
ax.set_ylabel('λ₁ × H*')
ax.set_title('λ₁ × H* vs k')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Cheeger bounds
ax = axes[1]
for N in N_VALUES:
    subset = [r for r in results if r['N'] == N]
    ks = [r['k'] for r in subset]
    h_lower = [r['h_lower_H'] for r in subset]
    h_upper = [r['h_upper_H'] for r in subset]
    ax.fill_between(ks, h_lower, h_upper, alpha=0.3, label=f'N={N}')
    ax.plot(ks, h_upper, 'o-')

ax.axhline(np.sqrt(26), color='blue', linestyle='--', alpha=0.7, label='√(2×13)')
ax.axhline(np.sqrt(28), color='red', linestyle='--', alpha=0.7, label='√(2×14)')
ax.set_xlabel('k (neighbors)')
ax.set_ylabel('h × H*')
ax.set_title('Cheeger bounds h × H*')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Stability check - λ₁ vs N at fixed k
ax = axes[2]
for k in K_VALUES:
    subset = [r for r in results if r['k'] == k]
    Ns = [r['N'] for r in subset]
    l1s = [r['lambda1_H'] for r in subset]
    ax.plot(Ns, l1s, 'o-', label=f'k={k}')

ax.axhline(14, color='red', linestyle='--', alpha=0.7)
ax.axhline(13, color='blue', linestyle='--', alpha=0.7)
ax.set_xlabel('N (samples)')
ax.set_ylabel('λ₁ × H*')
ax.set_title('Convergence with N')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('K7_Cheeger_Analysis.png', dpi=150)
plt.show()
print("✓ Saved: K7_Cheeger_Analysis.png")

In [ ]:
# Save results

output = {
    'metadata': {
        'notebook': 'K7_Cheeger_Computation.ipynb',
        'timestamp': datetime.now().isoformat(),
        'H_star': H_STAR,
        'N_values': N_VALUES,
        'k_values': K_VALUES
    },
    'theory': {
        'cheeger_inequality': 'λ₁ ≥ h²/4',
        'buser_inequality': 'λ₁ ≤ 2h (approx for graphs)',
        'expected_13': {'lambda1_H': 13, 'h_H': float(np.sqrt(26))},
        'expected_14': {'lambda1_H': 14, 'h_H': float(np.sqrt(28))}
    },
    'results': results
}

with open('K7_Cheeger_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("\n✓ Saved: K7_Cheeger_results.json")

---

## Conclusions

### What we learned:

1. **λ₁×H* depends on k** — This is the fundamental problem

2. **Cheeger bounds are derived from λ₁** — They don't help distinguish 13 vs 14

3. **The stability question**: Does h×H* stabilize as N→∞ for fixed k?

### Key insight:

The Cheeger constant h is **derived from λ₁** via spectral methods. 
Computing h directly (via isoperimetric ratio) requires the actual manifold geometry, 
not a graph approximation.

### Next steps:

1. **Analytical Cheeger**: Use TCS geometry (neck, volumes) to bound h analytically
2. **FEM methods**: Discretize the manifold with finite elements, not random sampling
3. **Accept the ambiguity**: Maybe λ₁×H* ∈ [13, 14] is the physical answer

---

*GIFT Spectral Gap Research — Cheeger Analysis*